In [1]:
import pathlib
import collections

## part 1 ##

In [2]:
testdata = '''1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581'''.splitlines()

In [3]:
puzzledata = pathlib.Path('day15.txt').read_text().splitlines()

In [4]:
len(puzzledata), len(puzzledata[0])

(100, 100)

In [5]:
def get_vertices(lines):
    v = {}
    for row, line in enumerate(lines):
        for col, c in enumerate(line):
            v[(row, col)] = int(c)
    return v

In [6]:
def get_adjacencies(vertices):
    adj = collections.defaultdict(list)
    for (row, col) in vertices:
        for nbr in [(row+1, col), (row-1, col), (row, col-1), (row, col+1)]:
            if nbr in vertices:
                adj[(row,col)].append(nbr)
    return adj

In [7]:
def get_graph(vertices):
    g = collections.defaultdict(dict)
    for (row, col) in vertices:
        for nbr in [(row+1, col), (row-1, col), (row, col-1), (row, col+1)]:
            if nbr in vertices:
                g[(row,col)][nbr] = vertices[nbr]
    return g

In [8]:
# from https://bradfieldcs.com/algos/graphs/dijkstras-algorithm/
import heapq


def calculate_distances(graph, starting_vertex):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[starting_vertex] = 0

    pq = [(0, starting_vertex)]
    while len(pq) > 0:
        current_distance, current_vertex = heapq.heappop(pq)

        # Nodes can get added to the priority queue multiple times. We only
        # process a vertex the first time we remove it from the priority queue.
        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            # Only consider this new path if it's better than any path we've
            # already found.
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))

    return distances

In [9]:
testv = get_vertices(testdata)
testg = get_graph(testv)
test_distances = calculate_distances(testg, (0,0))

In [10]:
test_distances[(9,9)]

40

In [11]:
puzzlev = get_vertices(puzzledata)
puzzleg = get_graph(puzzlev)

In [12]:
puzzle_distances = calculate_distances(puzzleg, (0,0))
puzzle_distances[(99,99)]

720

## part 2 ##

In [13]:
import numpy as np

In [14]:
def get_tile(data):
    nrows, ncols = len(data), len(data[0])
    tile = np.zeros((nrows, ncols), int)
    for i,row in enumerate(data):
        for j, c in enumerate(row):
            tile[i,j] = int(c)
    return tile

In [15]:
def get_offset_tile(tile, offset):
    newtile = tile.copy()
    newtile += offset
    rows, cols = newtile.shape
    for row in range(rows):
        for col in range(cols):
            if newtile[row, col] > 9:
                newtile[row, col] -= 9
    return newtile

In [16]:
def get_fullgrid(tile):
    num_tiles = 5
    tilerows, tilecols = tile.shape
    gridrows, gridcols = num_tiles*tilerows, num_tiles*tilecols
    grid = np.zeros((gridrows, gridcols), int)
    for rtile in range(num_tiles):
        for ctile in range(num_tiles):
            offset = rtile + ctile # taxicab distance from (0,0)
            offset_tile = get_offset_tile(tile, offset)
            grid[tilerows*rtile:tilerows*(rtile+1), 
                 tilecols*ctile:tilecols*(ctile+1)] = offset_tile
    return grid           

In [17]:
def get_grid_vertices(grid):
    rows, cols = grid.shape
    v = {}
    for i in range(rows):
        for j in range(cols):
            v[(i,j)] = grid[i,j]
    return v

In [18]:
def solve(data):
    tile = get_tile(data)
    grid = get_fullgrid(tile)
    rows, cols = grid.shape
    distances = calculate_distances(get_graph(get_grid_vertices(grid)), (0,0))
    return distances[(rows-1, cols-1)]

In [19]:
solve(testdata)

315

In [20]:
solve(puzzledata)

3025